### Introduction

* The dataset has been obtained from Kaggle.net.
* The dataset contains ratings information for 1895 games, alongwith 34 ESRB rating content descriptors.


In [1]:
import pandas as pd
import numpy as np

In [2]:
# import test and train dataset
# combining them for data wrangling

url1 = 'https://raw.githubusercontent.com/Belalsid94/VideoGameRatings_Capstone/main/Data/Video_games_esrb_rating.csv'
url2 = 'https://raw.githubusercontent.com/Belalsid94/VideoGameRatings_Capstone/main/Data/test_esrb.csv'

df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)

df = pd.concat([df1, df2], ignore_index=True)

df.head()

,title,console,alcohol_reference,animated_blood,blood,blood_and_gore,cartoon_violence,crude_humor,drug_reference,fantasy_violence,...,sexual_content,sexual_themes,simulated_gambling,strong_janguage,strong_sexual_content,suggestive_themes,use_of_alcohol,use_of_drugs_and_alcohol,violence,esrb_rating
0,Monster Jam Steel Titans 2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
1,Subnautica: Below Zero,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ET
2,NIER REPLICANT VER.1.22474487139…,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,M
3,Jamestown+,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,ET
4,Neptunia Virtual Stars,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,T


#### Data Assessment and Quality Check

In [3]:
# check data shape

df.shape

(2395, 34)

In [4]:
# check data types

df.dtypes

title                       object
console                      int64
alcohol_reference            int64
animated_blood               int64
blood                        int64
blood_and_gore               int64
cartoon_violence             int64
crude_humor                  int64
drug_reference               int64
fantasy_violence             int64
intense_violence             int64
language                     int64
lyrics                       int64
mature_humor                 int64
mild_blood                   int64
mild_cartoon_violence        int64
mild_fantasy_violence        int64
mild_language                int64
mild_lyrics                  int64
mild_suggestive_themes       int64
mild_violence                int64
no_descriptors               int64
nudity                       int64
partial_nudity               int64
sexual_content               int64
sexual_themes                int64
simulated_gambling           int64
strong_janguage              int64
strong_sexual_conten

In [5]:
# check missing values

df.isna().sum()

title                       0
console                     0
alcohol_reference           0
animated_blood              0
blood                       0
blood_and_gore              0
cartoon_violence            0
crude_humor                 0
drug_reference              0
fantasy_violence            0
intense_violence            0
language                    0
lyrics                      0
mature_humor                0
mild_blood                  0
mild_cartoon_violence       0
mild_fantasy_violence       0
mild_language               0
mild_lyrics                 0
mild_suggestive_themes      0
mild_violence               0
no_descriptors              0
nudity                      0
partial_nudity              0
sexual_content              0
sexual_themes               0
simulated_gambling          0
strong_janguage             0
strong_sexual_content       0
suggestive_themes           0
use_of_alcohol              0
use_of_drugs_and_alcohol    0
violence                    0
esrb_ratin

In [6]:
# drop rows with missing titles

df.dropna(inplace=True)

In [7]:
# check if the int64 variables have only 0's and 1's

int_val = pd.DataFrame()
int_val = df.drop(["title", "esrb_rating"], axis=1)

print(np.unique(int_val))

del int_val

[0 1]


In [8]:
# check column names 

list(df.columns)

['title',
 'console',
 'alcohol_reference',
 'animated_blood',
 'blood',
 'blood_and_gore',
 'cartoon_violence',
 'crude_humor',
 'drug_reference',
 'fantasy_violence',
 'intense_violence',
 'language',
 'lyrics',
 'mature_humor',
 'mild_blood',
 'mild_cartoon_violence',
 'mild_fantasy_violence',
 'mild_language',
 'mild_lyrics',
 'mild_suggestive_themes',
 'mild_violence',
 'no_descriptors',
 'nudity',
 'partial_nudity',
 'sexual_content',
 'sexual_themes',
 'simulated_gambling',
 'strong_janguage',
 'strong_sexual_content',
 'suggestive_themes',
 'use_of_alcohol',
 'use_of_drugs_and_alcohol',
 'violence',
 'esrb_rating']

In [9]:
# correct typo in the field: strong_language

df.rename(columns={'strong_janguage': 'strong_language'}, inplace=True)

In [10]:
# check if 'title' has any duplicate entries

df['title'].duplicated().any()

True

In [11]:
# check if target field has unique, categorical entries

df['esrb_rating'].unique()

array(['E', 'ET', 'M', 'T'], dtype=object)

#### Dropping corrupt/missing data

In [12]:
# check no descriptors column

df0 = df[df['no_descriptors'] == 0] # games should have some adult content
df1 = df[df['no_descriptors'] == 1] # games should have no adult content

# case 1: no_descriptors: 0, but games do not have any adult content

case1 = df0[~(df0.drop(['console', 'no_descriptors'], axis = 1) == 1).any(axis=1)]
print(case1['esrb_rating'].describe())

# case 2: no_descriptors: 1, but games do contain adult content

case2 = df1[(df1.drop(['console', 'no_descriptors'], axis = 1) == 1).any(axis=1)]
print(case2['esrb_rating'].describe())

count     19
unique     2
top        E
freq      18
Name: esrb_rating, dtype: object
count     67
unique     4
top        T
freq      43
Name: esrb_rating, dtype: object


* **Case 1:** 19 games have no content descriptor flags, but are incorrectly flagged as '0' in no descriptors field.
* **Case 2:** 67 games have been flagged as having no descriptors, but they do contain descriptor flags.

Due to this disparity, it is better if this column is dropped.

In [13]:
# drop column 'no_descriptors'

df = df.drop('no_descriptors', axis=1)

In [14]:
# create a new list of columns 'content_desc', excluding 'title', 'console', and 'esrb_rating'

fields = list(df.columns)
content_desc = [field for field in fields if field not in ['title', 'console','esrb_rating']]

# create a new field which sums all values under content descriptor field for every game

df.insert(loc=df.columns.get_loc('esrb_rating'), column='sum_descriptors', value=df[content_desc].sum(axis=1))

In [15]:
# check if all games rated ET, T or M have content descriptor data

df[(df['esrb_rating'] != 'E') & (df['sum_descriptors'] == 0)]

,title,console,alcohol_reference,animated_blood,blood,blood_and_gore,cartoon_violence,crude_humor,drug_reference,fantasy_violence,...,sexual_themes,simulated_gambling,strong_language,strong_sexual_content,suggestive_themes,use_of_alcohol,use_of_drugs_and_alcohol,violence,sum_descriptors,esrb_rating
900,Project Highrise - Architect's Edition,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ET


* 1 record has a rating of ET, but has no adult content flags. This seems suspicious as for the game to have a rating of ET, it should have atleast one adult content flag. 
* Upon checking the game's page on the ESRB website, it mentions the game does contain visuals with adult content. (Source: https://www.esrb.org/ratings/35710/project-highrise-architects-edition/)
* Most likely, the record's adult content flag data has been deleted/not entered, hence the record should be deleted as well.

In [16]:
# drop record

df.drop(index=900, inplace=True)

# confirmation

df[(df['esrb_rating'] != 'E') & (df['sum_descriptors'] == 0)]

,title,console,alcohol_reference,animated_blood,blood,blood_and_gore,cartoon_violence,crude_humor,drug_reference,fantasy_violence,...,sexual_themes,simulated_gambling,strong_language,strong_sexual_content,suggestive_themes,use_of_alcohol,use_of_drugs_and_alcohol,violence,sum_descriptors,esrb_rating


In [17]:
# reset index

df.reset_index(drop=True, inplace=True)

#### Data Aggregation

In [18]:
# data aggregation by ratings (E, ET, M, T)

df.groupby('esrb_rating')['title'].count()

esrb_rating
E     516
ET    528
M     477
T     873
Name: title, dtype: int64

In [19]:
# data aggregation by console type (0 - PS4, 1 - PS4 and XBox)

psonly = df[df['console']==0]
psxbox = df[df['console']==1]

print(f"PS4 only: {len(psonly)}")
print(f"PS4 and Xbox: {len(psxbox)}")

PS4 only: 1247
PS4 and Xbox: 1147


In [20]:
df.shape

(2394, 34)

### Summary:

* Final dataset has **2394 rows and 34 columns**.
* **'no_descriptors'** field was dropped, due to inconsistency in data. 
* **'sum_descriptors'** field was added, which displays sum of all content descriptor fields for each game.
* 1 record for game titled **'Project Highrise - Architect's Edition'** was dropped because of missing data.

In [21]:
df.to_csv('esrb_dataset_clean.csv', index=False)